In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score


In [ ]:
class MLApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Diabetes Prediction Model")

        self.label = tk.Label(root, text="Diabetes Prediction using K-NN", font=("Helvetica", 16))
        self.label.pack(pady=10)

        self.load_button = tk.Button(root, text="Load Dataset", command=self.load_dataset)
        self.load_button.pack(pady=5)

        self.train_button = tk.Button(root, text="Train Model", command=self.train_model)
        self.train_button.pack(pady=5)

        self.result_text = tk.Text(root, height=15, width=70)
        self.result_text.pack(pady=10)

        self.input_frame = tk.Frame(root)
        self.input_frame.pack(pady=10)

        self.glucose_label = tk.Label(self.input_frame, text="Glucose Level:")
        self.glucose_label.grid(row=0, column=0, padx=5, pady=5)
        self.glucose_entry = tk.Entry(self.input_frame)
        self.glucose_entry.grid(row=0, column=1, padx=5, pady=5)

        self.bloodpressure_label = tk.Label(self.input_frame, text="Blood Pressure Level:")
        self.bloodpressure_label.grid(row=1, column=0, padx=5, pady=5)
        self.bloodpressure_entry = tk.Entry(self.input_frame)
        self.bloodpressure_entry.grid(row=1, column=1, padx=5, pady=5)

        self.predict_button = tk.Button(root, text="Predict", command=self.predict_diabetes)
        self.predict_button.pack(pady=5)

    def load_dataset(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.dataset = pd.read_csv(file_path)
            messagebox.showinfo("Info", "Dataset loaded successfully!")
        else:
            messagebox.showwarning("Warning", "No file selected")

    def train_model(self):
        if not hasattr(self, 'dataset'):
            messagebox.showwarning("Warning", "Please load a dataset first")
            return

        dataset = self.dataset
        zero_not_accepted = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'Insulin']
        for column in zero_not_accepted:
            dataset[column] = dataset[column].replace(0, np.NaN)
            mean = int(dataset[column].mean(skipna=True))
            dataset[column] = dataset[column].replace(np.NaN, mean)

        # Split dataset into features (X) and target (y)
        X = dataset.iloc[:, 0:8]
        y = dataset.iloc[:, 8]

        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

        # Feature scaling
        self.sc_X = StandardScaler()
        X_train = self.sc_X.fit_transform(X_train)
        X_test = self.sc_X.transform(X_test)

        # Define the model: Initialize K-NN
        self.classifier = KNeighborsClassifier(n_neighbors=11, p=2, metric='euclidean')

        # Fit the model
        self.classifier.fit(X_train, y_train)

        # Predict the test set results
        y_pred = self.classifier.predict(X_test)

        # Evaluate the model
        cm = confusion_matrix(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        accuracy = accuracy_score(y_test, y_pred)

        # Display results
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, "Model trained successfully!\n\n")
        self.result_text.insert(tk.END, "Confusion Matrix:\n")
        self.result_text.insert(tk.END, f"{cm}\n\n")
        self.result_text.insert(tk.END, f"F1 Score: {f1}\n")
        self.result_text.insert(tk.END, f"Accuracy Score: {accuracy}\n")

    def predict_diabetes(self):
        try:
            glucose_level = float(self.glucose_entry.get())
            blood_pressure_level = float(self.bloodpressure_entry.get())
        except ValueError:
            messagebox.showwarning("Warning", "Please enter valid numbers for glucose and blood pressure levels")
            return

        if not hasattr(self, 'classifier'):
            messagebox.showwarning("Warning", "Please train the model first")
            return

        input_data = np.array([[glucose_level, blood_pressure_level, 0, 0, 0, 0, 0, 0]])  # Dummy values for other features
        input_data = self.sc_X.transform(input_data)
        prediction = self.classifier.predict(input_data)

        if prediction[0] == 1:
            result = "The model predicts that you have diabetes."
        else:
            result = "The model predicts that you do not have diabetes."

        messagebox.showinfo("Prediction Result", result)


In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = MLApp(root)
    root.mainloop()